## Representação do problema


In [8]:
import pandas as pd
import os

# Passo 1: Verificar os arquivos disponíveis no Colab
os.listdir('/content/')  # Isso vai listar todos os arquivos na pasta '/content/'

# Passo 2: Defina o nome do arquivo que você quer ler (substitua pelo nome do seu arquivo)
file_name = 'autopecas_motor_planilha.xlsx'  # Substitua pelo nome correto do arquivo que está no Colab

# Passo 3: Ler o arquivo XLSX
file_path = f'/content/{file_name}'

# Verificando se o arquivo realmente existe
if os.path.exists(file_path):
    # Ler o arquivo XLSX
    df = pd.read_excel(file_path)

    # Passo 4: Exibir as linhas do DataFrame
    for index, row in df.iterrows():
        print(f"Peça {index + 1}:")
        print(f"  Código: {row['Código']}")
        print(f"  Descrição: {row['Descrição']}")
        print(f"  Marca: {row['Marca']}")
        print(f"  Lote: {row['Lote']}")
        print(f"  Garantia: {row['Garantia']}")
        print("-" * 40)
else:
    print(f"Arquivo '{file_name}' não encontrado no diretório '/content/'.")


Peça 1:
  Código: M0001
  Descrição: Virabrequim
  Marca: MarcaC
  Lote: L196
  Garantia: 20 meses
----------------------------------------
Peça 2:
  Código: M0002
  Descrição: Tucho
  Marca: MarcaD
  Lote: L205
  Garantia: 23 meses
----------------------------------------
Peça 3:
  Código: M0003
  Descrição: Sensor de Pressão
  Marca: MarcaD
  Lote: L126
  Garantia: 25 meses
----------------------------------------
Peça 4:
  Código: M0004
  Descrição: Comando de Válvulas
  Marca: MarcaB
  Lote: L168
  Garantia: 34 meses
----------------------------------------
Peça 5:
  Código: M0005
  Descrição: Corrente de Comando
  Marca: MarcaA
  Lote: L108
  Garantia: 22 meses
----------------------------------------
Peça 6:
  Código: M0006
  Descrição: Comando de Válvulas
  Marca: MarcaC
  Lote: L127
  Garantia: 29 meses
----------------------------------------
Peça 7:
  Código: M0007
  Descrição: Válvula de Admissão
  Marca: MarcaD
  Lote: L135
  Garantia: 22 meses
-----------------------------

In [ ]:
import pandas as pd
import os

def verificar_marcas_mais_acionadas_garantia(file_name):
    # Verifica se o arquivo existe
    file_path = f'/content/{file_name}'

    if not os.path.exists(file_path):
        print(f"Arquivo '{file_name}' não encontrado no diretório '/content/'.")
        return

    # Carrega o arquivo XLSX
    df = pd.read_excel(file_path)

    # Verifica se a coluna 'Garantia' existe no DataFrame
    if 'Garantia' not in df.columns:
        print("A coluna 'Garantia' não foi encontrada no arquivo.")
        return

    # Filtra as linhas onde a Garantia não está vazia
    garantia_acionada = df[df['Garantia'].notna()]
    total_pecas = len(df)

    # Contagem de ocorrências das marcas
    marcas_contagem = garantia_acionada['Marca'].value_counts()

    # Exibe as marcas mais acionadas
    if marcas_contagem.empty:
        print("Não há registros de garantia acionada.")
    else:
        print("Marcas mais acionadas para garantia:")
        for marca, quantidade in marcas_contagem.items():
            print(f"{marca}: {quantidade} retornos de garantia")

        # Exibe o total de peças
        print(f"\nTotal de peças na planilha: {total_pecas}")

In [ ]:
# Exemplo de uso da função:
file_name = 'autopecas_motor_planilha.xlsx'  # Substitua pelo nome do arquivo no Colab
verificar_marcas_mais_acionadas_garantia(file_name)

Marcas mais acionadas para garantia:
MarcaD: 35 retornos de garantia
MarcaC: 33 retornos de garantia
MarcaB: 32 retornos de garantia
MarcaA: 26 retornos de garantia
MarcaE: 24 retornos de garantia

Total de peças na planilha: 150


## Fitness function


In [4]:
import pandas as pd
import os

def fitness_function(file_name, objetivo_retorno_min=50, objetivo_retorno_max=100):
    # Verifica se o arquivo existe no diretório
    file_path = f'/content/{file_name}'

    if not os.path.exists(file_path):
        print(f"Arquivo '{file_name}' não encontrado no diretório '/content/'.")
        return None

    # Carregar o arquivo XLSX
    df = pd.read_excel(file_path)

    # Verificar se as colunas necessárias existem no DataFrame
    if 'Marca' not in df.columns or 'Retornos de Garantia (Mensal)' not in df.columns:
        print("As colunas necessárias não foram encontradas no arquivo.")
        return None

    # Converter a coluna de 'Retornos de Garantia (Mensal)' para número inteiro
    df['Retornos de Garantia (Mensal)'] = pd.to_numeric(df['Retornos de Garantia (Mensal)'], errors='coerce')

    # Criar um DataFrame vazio para armazenar os retornos por marca e mês
    retornos_por_marca_mes = []

    # Iterar sobre cada linha para organizar os retornos por marca e mês
    for index, row in df.iterrows():
        mes = pd.to_datetime('today').replace(day=1)  # Primeiro dia do mês atual
        marca = row['Marca']
        quantidade_retorno = row['Retornos de Garantia (Mensal)']

        # Adiciona para o mês atual
        retornos_por_marca_mes.append([marca, mes, quantidade_retorno])

    # Criar um DataFrame com os retornos por marca e mês
    retornos_df = pd.DataFrame(retornos_por_marca_mes, columns=['Marca', 'Mes', 'Retornos'])

    # Garantir que 'Mes' esteja no formato datetime para agrupamento adequado
    retornos_df['Mes'] = pd.to_datetime(retornos_df['Mes']).dt.to_period('M')

    # Agrupar por 'Marca' e 'Mes' e somar os retornos de garantia (quantidade de peças retornadas)
    resultados = retornos_df.groupby(['Marca', 'Mes'])['Retornos'].sum().unstack(fill_value=0)

    # Se não houver dados
    if resultados.empty:
        print("Não há registros de retorno de garantia.")
        return None

    # Relatório detalhado sobre os resultados
    relatorio = []

    # Avaliar cada valor de retorno de garantia
    for marca in resultados.index:
        for mes in resultados.columns:
            quantidade_retorno = resultados.loc[marca, mes]

            # Criar a avaliação para cada marca/mês
            if objetivo_retorno_min <= quantidade_retorno <= objetivo_retorno_max:
                status = "Dentro do intervalo ideal"
                pontuacao = f"Pontuação +1 (Retorno de garantia {quantidade_retorno} dentro do intervalo {objetivo_retorno_min} a {objetivo_retorno_max})"
            else:
                status = "Fora do intervalo ideal"
                pontuacao = f"Pontuação -1 (Retorno de garantia {quantidade_retorno} fora do intervalo {objetivo_retorno_min} a {objetivo_retorno_max})"

            # Adiciona o resultado detalhado no relatório
            relatorio.append({
                'Marca': marca,
                'Mês': str(mes),  # Convertendo para string para facilitar a visualização
                'Quantidade de Retornos': quantidade_retorno,
                'Status': status,
                'Pontuação': pontuacao
            })

    # Mostrar o relatório detalhado
    for item in relatorio:
        print(f"Marca: {item['Marca']}, Mês: {item['Mês']}, Quantidade de Retornos: {item['Quantidade de Retornos']}")
        print(f"Status: {item['Status']}")
        print(f"Detalhamento: {item['Pontuação']}")
        print("-" * 80)

    # Calcular a pontuação total
    fitness_total = sum(1 if 'Dentro do intervalo ideal' in item['Status'] else -1 for item in relatorio)

    return fitness_total

    # Exemplo de uso:
file_name = 'autopecas_motor_planilha.xlsx'  # Substitua pelo nome do seu arquivo
fitness = fitness_function(file_name, objetivo_retorno_min=50, objetivo_retorno_max=80)

if fitness is not None:
    print(f"A pontuação de fitness dos retornos de garantia é: {fitness}")

Marca: MarcaA, Mês: 2024-11, Quantidade de Retornos: 56
Status: Dentro do intervalo ideal
Detalhamento: Pontuação +1 (Retorno de garantia 56 dentro do intervalo 50 a 80)
--------------------------------------------------------------------------------
Marca: MarcaB, Mês: 2024-11, Quantidade de Retornos: 96
Status: Fora do intervalo ideal
Detalhamento: Pontuação -1 (Retorno de garantia 96 fora do intervalo 50 a 80)
--------------------------------------------------------------------------------
Marca: MarcaC, Mês: 2024-11, Quantidade de Retornos: 82
Status: Fora do intervalo ideal
Detalhamento: Pontuação -1 (Retorno de garantia 82 fora do intervalo 50 a 80)
--------------------------------------------------------------------------------
Marca: MarcaD, Mês: 2024-11, Quantidade de Retornos: 88
Status: Fora do intervalo ideal
Detalhamento: Pontuação -1 (Retorno de garantia 88 fora do intervalo 50 a 80)
--------------------------------------------------------------------------------
Marca: M

## Hill climb


In [6]:
import pandas as pd
import os
import random

def fitness_function(file_name, objetivo_retorno_min=50, objetivo_retorno_max=100):
    """
    Avalia a qualidade dos retornos de garantia por marca e mês.
    - A função compara os retornos com os valores ideais e retorna uma pontuação de fitness.

    :param file_name: Nome do arquivo Excel com os dados de retornos de garantia.
    :param objetivo_retorno_min: Valor mínimo ideal de retornos de garantia por mês.
    :param objetivo_retorno_max: Valor máximo ideal de retornos de garantia por mês.
    :return: A pontuação de fitness total para os retornos de garantia.
    """
    # Verifica se o arquivo existe no diretório
    file_path = f'/content/{file_name}'

    if not os.path.exists(file_path):
        print(f"Arquivo '{file_name}' não encontrado no diretório '/content/'.")
        return None

    # Carregar o arquivo XLSX
    df = pd.read_excel(file_path)

    # Verificar se as colunas necessárias existem no DataFrame
    if 'Marca' not in df.columns or 'Retornos de Garantia (Mensal)' not in df.columns:
        print("As colunas necessárias não foram encontradas no arquivo.")
        return None

    # Converter a coluna de 'Retornos de Garantia (Mensal)' para número inteiro
    df['Retornos de Garantia (Mensal)'] = pd.to_numeric(df['Retornos de Garantia (Mensal)'], errors='coerce')

    # Criar um DataFrame vazio para armazenar os retornos por marca e mês
    retornos_por_marca_mes = []

    # Iterar sobre cada linha para organizar os retornos por marca e mês
    for index, row in df.iterrows():
        mes = pd.to_datetime('today').replace(day=1)  # Primeiro dia do mês atual
        marca = row['Marca']
        quantidade_retorno = row['Retornos de Garantia (Mensal)']

        # Adiciona para o mês atual
        retornos_por_marca_mes.append([marca, mes, quantidade_retorno])

    # Criar um DataFrame com os retornos por marca e mês
    retornos_df = pd.DataFrame(retornos_por_marca_mes, columns=['Marca', 'Mes', 'Retornos'])

    # Garantir que 'Mes' esteja no formato datetime para agrupamento adequado
    retornos_df['Mes'] = pd.to_datetime(retornos_df['Mes']).dt.to_period('M')

    # Agrupar por 'Marca' e 'Mes' e somar os retornos de garantia (quantidade de peças retornadas)
    resultados = retornos_df.groupby(['Marca', 'Mes'])['Retornos'].sum().unstack(fill_value=0)

    # Se não houver dados
    if resultados.empty:
        print("Não há registros de retorno de garantia.")
        return None

    # Relatório detalhado sobre os resultados
    relatorio = []

    # Avaliar cada valor de retorno de garantia
    for marca in resultados.index:
        for mes in resultados.columns:
            quantidade_retorno = resultados.loc[marca, mes]

            # Criar a avaliação para cada marca/mês
            if objetivo_retorno_min <= quantidade_retorno <= objetivo_retorno_max:
                status = "Dentro do intervalo ideal"
                pontuacao = 1  # Pontuação +1
            else:
                status = "Fora do intervalo ideal"
                pontuacao = -1  # Pontuação -1

            # Adiciona o resultado detalhado no relatório
            relatorio.append({
                'Marca': marca,
                'Mês': str(mes),  # Convertendo para string para facilitar a visualização
                'Quantidade de Retornos': quantidade_retorno,
                'Status': status,
                'Pontuação': pontuacao
            })

    # Calcular a pontuação total
    fitness_total = sum(item['Pontuação'] for item in relatorio)

    return fitness_total, relatorio

def hill_climbing(file_name, objetivo_retorno_min=50, objetivo_retorno_max=100, max_iter=100):
    """
    Aplica o algoritmo de Hill Climbing para melhorar a pontuação de fitness
    baseado nos retornos de garantia.

    :param file_name: Nome do arquivo Excel com os dados de retornos de garantia.
    :param objetivo_retorno_min: Valor mínimo ideal de retornos de garantia por mês.
    :param objetivo_retorno_max: Valor máximo ideal de retornos de garantia por mês.
    :param max_iter: Número máximo de iterações.
    :return: A melhor pontuação de fitness encontrada.
    """
    # Obter a pontuação inicial
    melhor_fitness, relatorio_atual = fitness_function(file_name, objetivo_retorno_min, objetivo_retorno_max)
    print(f"Pontuação inicial de fitness: {melhor_fitness}")

    for iteracao in range(max_iter):
        print(f"Iteração {iteracao + 1}/{max_iter}...")

        # Criar uma vizinhança para modificar
        # Aqui estamos aleatoriamente ajustando o número de retornos para uma marca
        nova_vizinhança = relatorio_atual.copy()
        marca_a_modificar = random.choice(nova_vizinhança)  # Escolher uma marca aleatória
        marca_a_modificar['Quantidade de Retornos'] = random.randint(30, 150)  # Ajuste aleatório

        # Calcular a pontuação de fitness da nova vizinhança
        novos_fitness, _ = fitness_function(file_name, objetivo_retorno_min, objetivo_retorno_max)

        # Comparar com a melhor pontuação
        if novos_fitness > melhor_fitness:
            melhor_fitness = novos_fitness
            relatorio_atual = nova_vizinhança  # Atualiza a vizinhança

        # Se não houver melhoria, podemos parar
        if novos_fitness == melhor_fitness:
            break

    return melhor_fitness

# Exemplo de uso
file_name = 'autopecas_motor_planilha.xlsx'  # Substitua pelo nome do seu arquivo
fitness_resultado = hill_climbing(file_name, objetivo_retorno_min=50, objetivo_retorno_max=100, max_iter=100)
print(f"A melhor pontuação de fitness encontrada é: {fitness_resultado}")


Pontuação inicial de fitness: 5
Iteração 1/100...
A melhor pontuação de fitness encontrada é: 5


## Simulated annealing


In [7]:
import pandas as pd
import os
import random
import math

def fitness_function(file_name, objetivo_retorno_min=50, objetivo_retorno_max=100):
    """
    Avalia a qualidade dos retornos de garantia por marca e mês.
    - A função compara os retornos com os valores ideais e retorna uma pontuação de fitness.

    :param file_name: Nome do arquivo Excel com os dados de retornos de garantia.
    :param objetivo_retorno_min: Valor mínimo ideal de retornos de garantia por mês.
    :param objetivo_retorno_max: Valor máximo ideal de retornos de garantia por mês.
    :return: A pontuação de fitness total para os retornos de garantia.
    """
    # Verifica se o arquivo existe no diretório
    file_path = f'/content/{file_name}'

    if not os.path.exists(file_path):
        print(f"Arquivo '{file_name}' não encontrado no diretório '/content/'.")
        return None

    # Carregar o arquivo XLSX
    df = pd.read_excel(file_path)

    # Verificar se as colunas necessárias existem no DataFrame
    if 'Marca' not in df.columns or 'Retornos de Garantia (Mensal)' not in df.columns:
        print("As colunas necessárias não foram encontradas no arquivo.")
        return None

    # Converter a coluna de 'Retornos de Garantia (Mensal)' para número inteiro
    df['Retornos de Garantia (Mensal)'] = pd.to_numeric(df['Retornos de Garantia (Mensal)'], errors='coerce')

    # Criar um DataFrame vazio para armazenar os retornos por marca e mês
    retornos_por_marca_mes = []

    # Iterar sobre cada linha para organizar os retornos por marca e mês
    for index, row in df.iterrows():
        mes = pd.to_datetime('today').replace(day=1)  # Primeiro dia do mês atual
        marca = row['Marca']
        quantidade_retorno = row['Retornos de Garantia (Mensal)']

        # Adiciona para o mês atual
        retornos_por_marca_mes.append([marca, mes, quantidade_retorno])

    # Criar um DataFrame com os retornos por marca e mês
    retornos_df = pd.DataFrame(retornos_por_marca_mes, columns=['Marca', 'Mes', 'Retornos'])

    # Garantir que 'Mes' esteja no formato datetime para agrupamento adequado
    retornos_df['Mes'] = pd.to_datetime(retornos_df['Mes']).dt.to_period('M')

    # Agrupar por 'Marca' e 'Mes' e somar os retornos de garantia (quantidade de peças retornadas)
    resultados = retornos_df.groupby(['Marca', 'Mes'])['Retornos'].sum().unstack(fill_value=0)

    # Se não houver dados
    if resultados.empty:
        print("Não há registros de retorno de garantia.")
        return None

    # Relatório detalhado sobre os resultados
    relatorio = []

    # Avaliar cada valor de retorno de garantia
    for marca in resultados.index:
        for mes in resultados.columns:
            quantidade_retorno = resultados.loc[marca, mes]

            # Criar a avaliação para cada marca/mês
            if objetivo_retorno_min <= quantidade_retorno <= objetivo_retorno_max:
                status = "Dentro do intervalo ideal"
                pontuacao = 1  # Pontuação +1
            else:
                status = "Fora do intervalo ideal"
                pontuacao = -1  # Pontuação -1

            # Adiciona o resultado detalhado no relatório
            relatorio.append({
                'Marca': marca,
                'Mês': str(mes),  # Convertendo para string para facilitar a visualização
                'Quantidade de Retornos': quantidade_retorno,
                'Status': status,
                'Pontuação': pontuacao
            })

    # Calcular a pontuação total
    fitness_total = sum(item['Pontuação'] for item in relatorio)

    return fitness_total, relatorio

def simulated_annealing(file_name, objetivo_retorno_min=50, objetivo_retorno_max=100, temp_inicial=1000, temp_final=1, alpha=0.95, max_iter=1000):
    """
    Aplica o algoritmo Simulated Annealing para melhorar a pontuação de fitness
    baseado nos retornos de garantia.

    :param file_name: Nome do arquivo Excel com os dados de retornos de garantia.
    :param objetivo_retorno_min: Valor mínimo ideal de retornos de garantia por mês.
    :param objetivo_retorno_max: Valor máximo ideal de retornos de garantia por mês.
    :param temp_inicial: Temperatura inicial.
    :param temp_final: Temperatura final.
    :param alpha: Taxa de resfriamento (quanto a temperatura diminui a cada iteração).
    :param max_iter: Número máximo de iterações.
    :return: A melhor pontuação de fitness encontrada.
    """

    # Obter a pontuação inicial
    melhor_fitness, relatorio_atual = fitness_function(file_name, objetivo_retorno_min, objetivo_retorno_max)
    print(f"Pontuação inicial de fitness: {melhor_fitness}")

    # Inicializa a temperatura
    temperatura = temp_inicial
    iteracao = 0

    while temperatura > temp_final and iteracao < max_iter:
        iteracao += 1
        print(f"Iteração {iteracao}/{max_iter}, Temperatura: {temperatura:.4f}")

        # Criar uma vizinhança para modificar (ajuste aleatório de uma marca)
        nova_vizinhança = relatorio_atual.copy()
        marca_a_modificar = random.choice(nova_vizinhança)  # Escolher uma marca aleatória
        marca_a_modificar['Quantidade de Retornos'] = random.randint(30, 150)  # Ajuste aleatório

        # Calcular a pontuação de fitness da nova vizinhança
        novos_fitness, _ = fitness_function(file_name, objetivo_retorno_min, objetivo_retorno_max)

        # Se a nova solução for melhor, aceita-a
        if novos_fitness > melhor_fitness:
            melhor_fitness = novos_fitness
            relatorio_atual = nova_vizinhança
        else:
            # Se for pior, aceita com uma probabilidade que depende da temperatura
            delta_fitness = novos_fitness - melhor_fitness
            probabilidade = math.exp(delta_fitness / temperatura)
            if random.random() < probabilidade:
                melhor_fitness = novos_fitness
                relatorio_atual = nova_vizinhança

        # Resfriar a temperatura
        temperatura *= alpha  # Diminui a temperatura

    return melhor_fitness

# Exemplo de uso:
file_name = 'autopecas_motor_planilha.xlsx'  # Substitua pelo nome do seu arquivo
fitness_resultado = simulated_annealing(file_name, objetivo_retorno_min=50, objetivo_retorno_max=100, temp_inicial=1000, temp_final=1, alpha=0.95, max_iter=1000)
print(f"A melhor pontuação de fitness encontrada é: {fitness_resultado}")


Pontuação inicial de fitness: 5
Iteração 1/1000, Temperatura: 1000.0000
Iteração 2/1000, Temperatura: 950.0000
Iteração 3/1000, Temperatura: 902.5000
Iteração 4/1000, Temperatura: 857.3750
Iteração 5/1000, Temperatura: 814.5062
Iteração 6/1000, Temperatura: 773.7809
Iteração 7/1000, Temperatura: 735.0919
Iteração 8/1000, Temperatura: 698.3373
Iteração 9/1000, Temperatura: 663.4204
Iteração 10/1000, Temperatura: 630.2494
Iteração 11/1000, Temperatura: 598.7369
Iteração 12/1000, Temperatura: 568.8001
Iteração 13/1000, Temperatura: 540.3601
Iteração 14/1000, Temperatura: 513.3421
Iteração 15/1000, Temperatura: 487.6750
Iteração 16/1000, Temperatura: 463.2912
Iteração 17/1000, Temperatura: 440.1267
Iteração 18/1000, Temperatura: 418.1203
Iteração 19/1000, Temperatura: 397.2143
Iteração 20/1000, Temperatura: 377.3536
Iteração 21/1000, Temperatura: 358.4859
Iteração 22/1000, Temperatura: 340.5616
Iteração 23/1000, Temperatura: 323.5335
Iteração 24/1000, Temperatura: 307.3569
Iteração 25/1000